# Cell 0: RAG 시스템 구축 (CPU, Pinecone + E5)

## 📋 개요
이 노트북은 **Retrieval-Augmented Generation (RAG)** 시스템을 구축하는 완전한 가이드입니다.

### 🎯 주요 기능
- **문서 처리**: 마크다운 파일에서 front-matter와 본문 분리
- **텍스트 청킹**: 500자 단위로 텍스트 분할 (75자 오버랩)
- **임베딩 생성**: E5-base-v2 모델로 벡터 임베딩 생성
- **벡터 저장**: Pinecone 서버리스 인덱스에 벡터 저장
- **하이브리드 검색**: 벡터 검색 + BM25 키워드 검색 결합
- **재랭킹**: CrossEncoder로 검색 결과 재정렬
- **답변 생성**: OpenAI GPT-4o-mini로 컨텍스트 기반 답변 생성

### 🔧 기술 스택
- **임베딩 모델**: `intfloat/e5-base-v2` (768차원)
- **벡터 DB**: Pinecone (서버리스)
- **재랭킹**: `cross-encoder/ms-marco-MiniLM-L-6-v2`
- **LLM**: OpenAI GPT-4o-mini
- **언어**: Python 3.11+ (CPU 전용)

### 📁 데이터 구조
```
문서 → Front-matter + 본문 → 청킹 → 임베딩 → Pinecone 저장
```

> **주의**: CPU 환경에서 실행되며, 한글 파일명은 자동으로 ASCII로 변환됩니다.


## Cell 1: 🔍 1. 환경 확인

Python 환경과 주요 라이브러리 버전을 확인합니다.

In [21]:
# Cell 2: 환경 확인 코드
import sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())

try:
    import torch
    print('Torch:', torch.__version__, '| CUDA 사용 여부:', torch.cuda.is_available())
except Exception as e:
    print('Torch 미설치 또는 오류:', e)

try:
    import transformers
    print('Transformers:', transformers.__version__)
except Exception as e:
    print('Transformers 미설치 또는 오류:', e)


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
Platform: Windows-10-10.0.26100-SP0
Torch: 2.8.0+cpu | CUDA 사용 여부: False
Transformers: 4.56.1


## Cell 3: 📦 2. 필수 패키지 설치

RAG 시스템에 필요한 핵심 패키지들을 설치합니다.

In [22]:
# Cell 4: 패키지 설치 코드
import sys
!{sys.executable} -m pip install -q --upgrade pip
!{sys.executable} -m pip install -q "pinecone>=5.0.0" sentence-transformers
print("설치 완료")


설치 완료


## Cell 5: 🗄️ 3. Pinecone 설정

### 사전 준비
1. [Pinecone](https://www.pinecone.io) 무료 계정 가입
2. API 키 발급 및 환경변수 설정: `PINECONE_API_KEY`
3. OpenAI API 키 설정: `OPENAI_API_KEY`

### 인덱스 생성
- **이름**: `rag-uv-chatbot`
- **차원**: 768 (E5-base-v2 출력 차원)
- **메트릭**: cosine similarity
- **클라우드**: AWS us-east-1


In [11]:
# Cell 6: Pinecone 인덱스 생성/연결
import os 
from pinecone import Pinecone, ServerlessSpec

# 🔑 PINECONE API KEY
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# 인덱스 이름 지정 (소문자, 하이픈만 허용)
index_name = "rag-univera-chatbot-v2"

# 인덱스 없으면 생성
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # multilingual-e5-large 출력 차원
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print("인덱스 생성 완료")
else:
    print("이미 인덱스 있음")


이미 인덱스 있음


## Cell 7: 🔧 4. ID 변환 (Pinecone 호환)

Pinecone은 벡터 ID에 ASCII 문자만 허용하므로, 한글 파일명을 ASCII로 변환합니다.

### 변환 예시
- `'M40X_마일드_클렌징_오일-front'` → `'m40x_maildeu_keulrenjing_oil-front'`
- `'한글_문서-chunk-0'` → `'hangeul_munseo-chunk-0'`


## Cell 8: 📊 5. 인덱스 상태 확인

Pinecone 인덱스의 현재 상태를 확인합니다.

In [12]:
# Cell 9: 인덱스 상태 확인
index = pc.Index(index_name)
print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 703}},
 'total_vector_count': 703,
 'vector_type': 'dense'}


## Cell 10: 📁 6. 문서 처리 파이프라인

### 처리 단계
1. **파일 수집**: 지정된 폴더에서 모든 `.md` 파일 수집
2. **텍스트 파싱**: Front-matter와 본문 분리
3. **텍스트 청킹**: 500자 단위로 분할 (75자 오버랩)
4. **메타데이터 추출**: 제목, 카테고리, 키워드 등 추출
5. **임베딩 생성**: E5-base-v2 모델로 벡터 생성
6. **벡터 저장**: Pinecone에 배치 업로드

## Cell 11: 📂 6.1 파일 수집

재귀적으로 모든 하위 폴더에서 마크다운 파일을 수집합니다.

In [13]:
# Cell 12: 파일 수집 코드
import os, re, json

# 새로운 폴더 경로
folder_path1 = r"C:\Users\admin\Desktop\문서\9999.DDP\RAG Database_Products Info (90 files)"
folder_path2 = r"C:\Users\admin\Desktop\문서\9999.DDP\RAG Database_Company Info (63 files)"

def find_md_files_direct(folder_path):
    """폴더에서 직접 .md 파일 찾기 (재귀 없음)"""
    md_files = []
    if os.path.exists(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith(".md"):
                md_files.append(os.path.join(folder_path, file))
    return md_files

# 두 폴더에서 마크다운 파일 수집
md_files1 = find_md_files_direct(folder_path1)
md_files2 = find_md_files_direct(folder_path2)

# 모든 마크다운 파일 합치기
md_files = md_files1 + md_files2

print("제품 정보 폴더 마크다운 파일 수:", len(md_files1))
print("회사 정보 폴더 마크다운 파일 수:", len(md_files2))
print("전체 마크다운 파일 수:", len(md_files))
print("예시:", md_files[:5])


제품 정보 폴더 마크다운 파일 수: 90
회사 정보 폴더 마크다운 파일 수: 63
전체 마크다운 파일 수: 153
예시: ['C:\\Users\\admin\\Desktop\\문서\\9999.DDP\\RAG Database_Products Info (90 files)\\M40X_마일드_클렌징_오일.md', 'C:\\Users\\admin\\Desktop\\문서\\9999.DDP\\RAG Database_Products Info (90 files)\\M40X_마일드_폼_클렌져.md', 'C:\\Users\\admin\\Desktop\\문서\\9999.DDP\\RAG Database_Products Info (90 files)\\New알로에버플러스.md', 'C:\\Users\\admin\\Desktop\\문서\\9999.DDP\\RAG Database_Products Info (90 files)\\W389_더마_브라이트닝.md', 'C:\\Users\\admin\\Desktop\\문서\\9999.DDP\\RAG Database_Products Info (90 files)\\W389_더마_브라이트닝_로션.md']


### 6.2 텍스트 파싱

마크다운 파일에서 front-matter와 본문을 분리합니다.

In [14]:
# Cell 14: 텍스트 파싱 코드 (단순화)
import yaml
import re

def parse_document_unified(text, filename):
    """Front-matter 파싱 함수 (단순화)"""
    
    # YAML front-matter 파싱
    fm_match = re.match(r"^---\s*\n(.*?)\n---\s*\n(.*)", text, re.DOTALL)
    if fm_match:
        front_matter_raw, body = fm_match.groups()
        try:
            fm_dict = yaml.safe_load(front_matter_raw)
            if fm_dict:
                print(f"✅ 파싱 완료 | {filename}")
                return fm_dict, body
        except yaml.YAMLError as e:
            print(f"❌ YAML 파싱 오류 | {filename} - {e}")
    
    # front-matter가 없으면 기본값 반환
    print(f"⚠️ Front-matter 없음 | {filename}")
    name_without_ext = os.path.splitext(filename)[0]
    title = name_without_ext.replace('_', ' ')
    
    fm_dict = {
        "title": title,
        "category1": "기타",
        "category2": "",
        "category3": "",
        "category4": "",
        "keywords": title
    }
    return fm_dict, text

docs = []
for file_path in md_files:
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    
    # 통합 파싱 함수 사용
    fm_dict, body = parse_document_unified(text, os.path.basename(file_path))
    
    docs.append({
        "id": os.path.basename(file_path),
        "front_matter": fm_dict,
        "body": body
    })

print("불러온 문서 수:", len(docs))


✅ 파싱 완료 | M40X_마일드_클렌징_오일.md
✅ 파싱 완료 | M40X_마일드_폼_클렌져.md
✅ 파싱 완료 | New알로에버플러스.md
✅ 파싱 완료 | W389_더마_브라이트닝.md
✅ 파싱 완료 | W389_더마_브라이트닝_로션.md
✅ 파싱 완료 | W389_더마_브라이트닝_선크림.md
✅ 파싱 완료 | W389_더마_브라이트닝_선크림_기획세트.md
✅ 파싱 완료 | W389_더마_브라이트닝_스팟_에센스.md
✅ 파싱 완료 | W389_더마_브라이트닝_크림.md
✅ 파싱 완료 | W389_더마_브라이트닝_토너.md
✅ 파싱 완료 | W389_더마_브라이트닝_필링젤.md
✅ 파싱 완료 | 그린칼슘플러스.md
✅ 파싱 완료 | 남양931플러스.md
✅ 파싱 완료 | 노회비책.md
✅ 파싱 완료 | 레벨지플러스.md
✅ 파싱 완료 | 리제니케어A.md
✅ 파싱 완료 | 멀티비타민_맥스.md
✅ 파싱 완료 | 메타번슬림핏.md
✅ 파싱 완료 | 미즈에버플러스.md
✅ 파싱 완료 | 베라힐.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_(바디로션,바디워시,샴푸,시크릿케어).md
✅ 파싱 완료 | 베라힐_내추럴_마일드_바디로션.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_바디미스트.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_바디워시.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_샴푸.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_시크릿케어.md
✅ 파싱 완료 | 베라힐_토탈케어_치약.md
✅ 파싱 완료 | 베라힐퍼퓸드핸드크림.md
✅ 파싱 완료 | 비움클렌즈.md
✅ 파싱 완료 | 빌리브뷰티콜라겐.md
✅ 파싱 완료 | 빌리브웰니스푸드S.md
✅ 파싱 완료 | 빌리브효소.md
✅ 파싱 완료 | 수면온.md
✅ 파싱 완료 | 슈퍼겔맥스.md
✅ 파싱 완료 | 아레지오플러스.md
✅ 파싱 완료 | 아보민플러스.md
✅ 파싱 완료 | 알로맥프로지.md
✅ 파싱 완료 | 알로신.md
✅ 파싱 완료 | 알로에바이오틱스.md
✅ 파싱 완료 | 알로에버플러스.md
✅ 파싱

# Cell 15: ⚠️ 주의: Pinecone 인덱스 초기화

## 🚨 **위험한 작업입니다!**

이 셀은 **Pinecone 인덱스의 모든 데이터를 영구적으로 삭제**합니다.

### ⚠️ **실행 전 확인사항**
- [ ] **백업이 필요한가요?** (현재 데이터를 보존해야 하는 경우)
- [ ] **정말로 모든 데이터를 삭제하시겠습니까?**
- [ ] **다른 사용자가 이 인덱스를 사용하고 있지 않나요?**

### 🔄 **언제 사용하나요?**
- 새로운 AI 기반 메타데이터로 데이터를 재처리할 때
- 인덱스에 문제가 생겨서 완전히 초기화해야 할 때
- 테스트용 데이터를 정리할 때

### ⚡ **실행 후 해야 할 일**
1. **Cell 1부터 순서대로 실행**하여 새로운 데이터로 재처리
2. **대화 시스템 테스트**로 정상 작동 확인

### 💡 **안전한 사용법**
```python
# 실행 전 현재 상태 확인
stats = index.describe_index_stats()
print(f"현재 벡터 수: {stats.total_vector_count}")

# 정말 삭제하시겠습니까? 확인 후 실행
```

---
**⚠️ 주의: 이 작업은 되돌릴 수 없습니다!**


In [20]:
# Cell 16: 🗑️ Pinecone 인덱스 초기화 (모든 데이터 삭제)
print("🗑️ Pinecone 인덱스의 모든 데이터를 삭제합니다...")

# 인덱스 연결
index = pc.Index(index_name)

# 현재 인덱스 상태 확인
stats = index.describe_index_stats()
print(f"현재 벡터 수: {stats.total_vector_count}")

# 모든 벡터 삭제
if stats.total_vector_count > 0:
    index.delete(delete_all=True)
    print("✅ 모든 데이터 삭제 완료!")
    
    # 삭제 후 상태 확인
    stats_after = index.describe_index_stats()
    print(f"삭제 후 벡터 수: {stats_after.total_vector_count}")
else:
    print("이미 비어있는 인덱스입니다.")


🗑️ Pinecone 인덱스의 모든 데이터를 삭제합니다...
현재 벡터 수: 901
✅ 모든 데이터 삭제 완료!
삭제 후 벡터 수: 0


## Cell 17: ✂️ 6.3 텍스트 청킹 및 메타데이터 생성

텍스트를 500자 단위로 분할하고 각 청크에 메타데이터를 추가합니다.

#### 청킹 설정
- **청크 크기**: 500자
- **오버랩**: 75자
- **메타데이터**: 제목, 카테고리, 키워드, 소스 문서 등

In [23]:
import json
import re
# Cell 18: 텍스트 청킹 및 메타데이터 생성 코드
from slugify import slugify

def chunk_text(text, chunk_size=1024, overlap=100):
    chunks, start = [], 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

def normalize_fm_value(v):
    if isinstance(v, list):
        return ", ".join(map(str, v))
    return v

def clean_id(filename):
    """파일명을 slugify로 정리 (Pinecone 호환)"""
    name_without_ext = os.path.splitext(filename)[0]
    clean_name = slugify(name_without_ext, separator='_')
    return clean_name if clean_name else "doc"

# 먼저 문서 파싱 실행 (docs 변수 생성)
print("📄 문서 파싱 시작...")
docs = []
for file_path in md_files:
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    
    # 통합 파싱 함수 사용
    fm_dict, body = parse_document_unified(text, os.path.basename(file_path))
    
    docs.append({
        "id": os.path.basename(file_path),
        "front_matter": fm_dict,
        "body": body
    })

print("불러온 문서 수:", len(docs))
print("첫 문서 front-matter:", docs[0]["front_matter"])

# 청킹 및 메타데이터 생성
print("✂️ 청킹 및 메타데이터 생성 시작...")
chunked_docs_meta = []
for doc in docs:
    fm = {k: normalize_fm_value(v) for k, v in (doc.get("front_matter") or {}).items()}
    base_meta = {
        "source_doc": doc["id"],
        "title": fm.get("title", ""),
        "category1": fm.get("category1", ""),
        "category2": fm.get("category2", ""),
        "category3": fm.get("category3", ""),
        "category4": fm.get("category4", ""),
        "keywords": fm.get("keywords", ""),
    }

    # ID 정리 (한글 유지)
    clean_doc_id = clean_id(doc["id"])

    # front-matter 도 하나의 레코드로
    if doc.get("front_matter"):
        fm_json = json.dumps(doc["front_matter"], ensure_ascii=False)
        fm_text = f"passage: {fm_json}"
        chunked_docs_meta.append({
            "id": f"{clean_doc_id}-front",
            "text": fm_text,
            "metadata": {**base_meta, "kind": "front", "text_content": fm_json[:40000]}
        })

    # 본문 청크
    for i, chunk in enumerate(chunk_text(doc["body"], chunk_size=500, overlap=75)):  # 500/75 설정 적용
        chunk_text_full = f"passage: {chunk}"
        chunked_docs_meta.append({
            "id": f"{clean_doc_id}-chunk-{i}",
            "text": chunk_text_full,
            "metadata": {**base_meta, "kind": "chunk", "text_content": chunk[:40000]}
        })

print("메타 포함 청크 수:", len(chunked_docs_meta))
print("예시:", chunked_docs_meta[0]["id"], "=>", chunked_docs_meta[0]["metadata"])


📄 문서 파싱 시작...
✅ 파싱 완료 | M40X_마일드_클렌징_오일.md
✅ 파싱 완료 | M40X_마일드_폼_클렌져.md
✅ 파싱 완료 | New알로에버플러스.md
✅ 파싱 완료 | W389_더마_브라이트닝.md
✅ 파싱 완료 | W389_더마_브라이트닝_로션.md
✅ 파싱 완료 | W389_더마_브라이트닝_선크림.md
✅ 파싱 완료 | W389_더마_브라이트닝_선크림_기획세트.md
✅ 파싱 완료 | W389_더마_브라이트닝_스팟_에센스.md
✅ 파싱 완료 | W389_더마_브라이트닝_크림.md
✅ 파싱 완료 | W389_더마_브라이트닝_토너.md
✅ 파싱 완료 | W389_더마_브라이트닝_필링젤.md
✅ 파싱 완료 | 그린칼슘플러스.md
✅ 파싱 완료 | 남양931플러스.md
✅ 파싱 완료 | 노회비책.md
✅ 파싱 완료 | 레벨지플러스.md
✅ 파싱 완료 | 리제니케어A.md
✅ 파싱 완료 | 멀티비타민_맥스.md
✅ 파싱 완료 | 메타번슬림핏.md
✅ 파싱 완료 | 미즈에버플러스.md
✅ 파싱 완료 | 베라힐.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_(바디로션,바디워시,샴푸,시크릿케어).md
✅ 파싱 완료 | 베라힐_내추럴_마일드_바디로션.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_바디미스트.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_바디워시.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_샴푸.md
✅ 파싱 완료 | 베라힐_내추럴_마일드_시크릿케어.md
✅ 파싱 완료 | 베라힐_토탈케어_치약.md
✅ 파싱 완료 | 베라힐퍼퓸드핸드크림.md
✅ 파싱 완료 | 비움클렌즈.md
✅ 파싱 완료 | 빌리브뷰티콜라겐.md
✅ 파싱 완료 | 빌리브웰니스푸드S.md
✅ 파싱 완료 | 빌리브효소.md
✅ 파싱 완료 | 수면온.md
✅ 파싱 완료 | 슈퍼겔맥스.md
✅ 파싱 완료 | 아레지오플러스.md
✅ 파싱 완료 | 아보민플러스.md
✅ 파싱 완료 | 알로맥프로지.md
✅ 파싱 완료 | 알로신.md
✅ 파싱 완료 | 알로에바이오틱스.md
✅ 파싱 완료 | 알

## Cell 19: 🚀 7. 모델 설치 및 임베딩 생성

E5-base-v2 모델을 로드하고 텍스트를 벡터로 변환합니다.

In [24]:
# Cell 20: 필요한 패키지 설치
import sys
!{sys.executable} -m pip install -q python-slugify rank-bm25
print("패키지 설치 완료")


패키지 설치 완료


In [25]:
# Cell 21: E5 모델 로드 및 임베딩 생성
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = "intfloat/multilingual-e5-large"
device = "cpu"
model = SentenceTransformer(model_name, device=device)

print(f"모델 로드 완료: {model_name}")
print(f"디바이스: {device}")
print(f"출력 차원: 1024")


모델 로드 완료: intfloat/multilingual-e5-large
디바이스: cpu
출력 차원: 1024


In [26]:
# Cell 22: 텍스트를 벡터로 변환하고 Pinecone에 업로드
print("🚀 벡터 임베딩 생성 중...")
texts = [d["text"] for d in chunked_docs_meta]
embs = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
print(f"✅ 임베딩 생성 완료: {len(embs)}개")

print("📦 벡터 레코드 준비 중...")
records = []
for i, d in enumerate(chunked_docs_meta):
    records.append({
        "id": d["id"],
        "values": embs[i].tolist(),
        "metadata": d["metadata"]
    })
print(f"✅ 레코드 준비 완료: {len(records)}개")

# 배치 업로드 (100개씩)
batch_size = 100
total_batches = (len(records) + batch_size - 1) // batch_size
print(f"📤 Pinecone 업로드 시작: {total_batches}개 배치")

for s in range(0, len(records), batch_size):
    batch_num = (s // batch_size) + 1
    batch_records = records[s:s+batch_size]
    
    print(f"  📦 배치 {batch_num}/{total_batches}: {len(batch_records)}개 벡터 업로드 중...", end=" ")
    index.upsert(vectors=batch_records)
    print("✅ 완료")

print(f"\n🎉 업로드 완료: {len(records)}개 벡터")
print("📊 인덱스 통계:")
print(index.describe_index_stats())


🚀 벡터 임베딩 생성 중...
✅ 임베딩 생성 완료: 901개
📦 벡터 레코드 준비 중...
✅ 레코드 준비 완료: 901개
📤 Pinecone 업로드 시작: 10개 배치
  📦 배치 1/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 2/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 3/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 4/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 5/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 6/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 7/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 8/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 9/10: 100개 벡터 업로드 중... ✅ 완료
  📦 배치 10/10: 1개 벡터 업로드 중... ✅ 완료

🎉 업로드 완료: 901개 벡터
📊 인덱스 통계:
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1081}},
 'total_vector_count': 1081,
 'vector_type': 'dense'}


## Cell 23: 🔍 8. 하이브리드 검색 시스템

벡터 검색과 BM25 키워드 검색을 결합한 하이브리드 검색을 구현합니다.

### 검색 전략
1. **벡터 검색**: E5 임베딩 기반 의미적 유사도 검색
2. **BM25 검색**: 키워드 기반 정확도 검색
3. **점수 융합**: 벡터 점수(60%) + BM25 점수(40%)
4. **재랭킹**: CrossEncoder로 최종 순위 조정

In [ ]:
# Cell 24: ⚠️ 중복 업로드 방지: 이미 Cell 22에서 업로드 완료됨
# 이 셀은 실행하지 마세요!

print("⚠️ 이미 Cell 22에서 업로드가 완료되었습니다.")
print("📊 현재 인덱스 상태:")
print(index.describe_index_stats())


⚠️ 이미 Cell 22에서 업로드가 완료되었습니다.
📊 현재 인덱스 상태:
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1082}},
 'total_vector_count': 1082,
 'vector_type': 'dense'}


## Cell 25: 🔍 8.1 BM25 검색 구현

키워드 기반 검색을 위한 BM25 인덱스를 구축합니다.

In [45]:
# Cell 26: Pinecone 기반 하이브리드 검색 (BM25는 Pinecone 후보에만 적용)
import re, numpy as np
from rank_bm25 import BM25Okapi

# 토크나이저
def simple_tokenize(s: str):
    return re.findall(r"[A-Za-z0-9가-힣]+", (s or "").lower())

# 벡터 검색: Pinecone에서 후보 생성 (메타데이터 포함)
def vector_search(query: str, top_k: int = 50, meta_filter=None):
    q_vec = model.encode([f"query: {query}"], convert_to_numpy=True, normalize_embeddings=True)[0]
    kwargs = {"vector": q_vec.tolist(), "top_k": top_k, "include_values": False, "include_metadata": True}
    if meta_filter:
        kwargs["filter"] = meta_filter
    res = index.query(**kwargs)
    # 반환: (id, vec_score, metadata)
    return [(m["id"], float(m["score"]), m.get("metadata", {})) for m in res.get("matches", [])]

# BM25 재점수화: Pinecone 후보 텍스트에 대해서만 적용
def bm25_over_candidates(query: str, candidates):
    # candidates: [(id, vec_score, metadata), ...]
    docs = []
    ids = []
    for cid, _, meta in candidates:
        text = (meta or {}).get("text_content") or ""
        if not text:
            # 텍스트가 없으면 제목/키워드 등으로 대체
            title = (meta or {}).get("title") or ""
            keywords = (meta or {}).get("keywords") or ""
            text = f"{title}\n{keywords}"
        ids.append(cid)
        docs.append(simple_tokenize(text))
    if not docs:
        return {}
    bm25 = BM25Okapi(docs)
    q_tokens = simple_tokenize(query)
    scores = bm25.get_scores(q_tokens) if q_tokens else np.zeros(len(ids))
    # 정규화 (최댓값 기준)
    max_b = float(np.max(scores)) if len(scores) else 0.0
    norm = [float(s) / max_b if max_b > 0 else 0.0 for s in scores]
    return {ids[i]: norm[i] for i in range(len(ids))}

# 하이브리드: vec 후보(top_k_vec) 생성 후 BM25로 재점수화, 가중합으로 최종 정렬
def hybrid_candidates(query: str, top_k_vec: int = 50, meta_filter=None, vec_weight: float = 0.7, bm25_weight: float = 0.3):
    vec_cands = vector_search(query, top_k=top_k_vec, meta_filter=meta_filter)
    if not vec_cands:
        return []
    bm25_scores = bm25_over_candidates(query, vec_cands)
    merged = []
    for cid, vscore, _meta in vec_cands:
        bscore = bm25_scores.get(cid, 0.0)
        final = vec_weight * float(vscore) + bm25_weight * float(bscore)
        merged.append((cid, final))
    merged.sort(key=lambda x: x[1], reverse=True)
    return merged

print("✅ Pinecone 전용 하이브리드 검색 준비 완료")

✅ Pinecone 전용 하이브리드 검색 준비 완료


## Cell 28: 🔍 8.2 검색 시스템

하이브리드 검색으로 관련 문서를 찾습니다.


In [46]:
# Cell 29: 검색 시스템 코드 (Pinecone 전용)
# 로컬 백업 제거, 항상 Pinecone에서만 조회

def get_text_from_pinecone(doc_id):
    """Pinecone에서 텍스트 내용 조회"""
    try:
        result = index.fetch(ids=[doc_id])
        if doc_id in result.vectors:
            metadata = result.vectors[doc_id].metadata or {}
            return metadata.get("text_content", "")
    except Exception:
        pass
    return ""

def get_meta_from_pinecone(doc_id):
    """Pinecone에서 메타데이터 조회"""
    try:
        result = index.fetch(ids=[doc_id])
        if doc_id in result.vectors:
            return result.vectors[doc_id].metadata or {}
    except Exception:
        pass
    return {}

# 단순 래퍼 (백업 없음)
class PineconeDict:
    def __init__(self, fetch_func):
        self.fetch_func = fetch_func
    
    def get(self, key, default=""):
        return self.fetch_func(key) or default
    
    def __getitem__(self, key):
        result = self.fetch_func(key)
        return result or ""

id2text = PineconeDict(get_text_from_pinecone)
id2meta = PineconeDict(get_meta_from_pinecone)

print("✅ Pinecone 전용 검색 시스템 준비 완료")


✅ Pinecone 전용 검색 시스템 준비 완료


## Cell 30: 🤖 9. 답변 생성 시스템

OpenAI GPT-4o-mini를 사용하여 컨텍스트 기반 답변을 생성합니다.

### 답변 생성 과정
1. **검색**: 하이브리드 검색으로 관련 문서 검색
2. **재랭킹**: CrossEncoder로 상위 5개 문서 선별
3. **컨텍스트 구성**: 선별된 문서들을 연결하여 컨텍스트 생성
4. **답변 생성**: GPT-4o-mini로 최종 답변 생성
5. **출처 표기**: 참고한 문서들을 bullet로 나열

In [51]:
# Cell 32: LLM 구동 RAG 챗봇 (Pinecone 전용, 하드코딩 없음)
import time
from collections import deque
from typing import List, Dict, Tuple

try:
    from openai import OpenAI
    _openai_client = OpenAI()
except Exception as _e:
    _openai_client = None

# 전제: Cell 26의 vector_search, bm25_over_candidates 가 이미 정의되어 있어야 함

# 동적 어휘/엔티티 수집 (Pinecone 메타데이터 기반)
def collect_vocab_entities(sample_k: int = 200) -> Tuple[List[str], List[str]]:
    try:
        res = index.query(vector=[0.0]*1024, top_k=sample_k, include_values=False, include_metadata=True)
    except Exception:
        return [], []
    vocab, entities = set(), set()
    for m in res.get("matches", []):
        md = m.get("metadata", {}) or {}
        title = (md.get("title") or "").strip()
        if title:
            vocab.add(title)
            entities.add(title)
        kws = md.get("keywords")
        if isinstance(kws, str):
            for k in kws.split(','):
                if k.strip():
                    vocab.add(k.strip())
        elif isinstance(kws, list):
            for k in kws:
                s = str(k).strip()
                if s:
                    vocab.add(s)
    return sorted(vocab), sorted(entities)

# LLM 유틸: JSON만 반환을 강제하는 간단 헬퍼
def llm_json(prompt: str, max_tokens: int = 300) -> Dict:
    if not _openai_client:
        return {}
    messages = [
        {"role": "system", "content": "항상 유효한 JSON만 출력하세요. 설명/문장 금지."},
        {"role": "user", "content": prompt}
    ]
    r = _openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.2,
        max_tokens=max_tokens
    )
    import json
    txt = r.choices[0].message.content
    try:
        return json.loads(txt)
    except Exception:
        return {}

# 의도 분류 (normal / count / followup) - 규칙 없이 LLM 판단
def classify_intent_llm(question: str, history: List[Tuple[str,str,str]]) -> str:
    prompt = f"""
    다음 한국어 질문의 의도를 분류하세요. 가능한 라벨: ["normal","count","followup"].
    - normal: 일반 정보 질의
    - count: 개수를 묻거나 총합 등 집계 질문
    - followup: 직전/이전 발화를 지시어/대명사로 참조하는 질문

    질문: "{question}"
    최근 대화(최대 3턴): {history}

    JSON으로만 출력:
    {{"intent":"normal|count|followup"}}
    """
    return llm_json(prompt, max_tokens=120).get("intent", "normal")

# 질의 재작성 + 키워드 확장 + 대명사 해소 (LLM)
def rewrite_and_expand_llm(question: str, history: List[Tuple[str,str,str]], vocab: List[str], entities: List[str]) -> Tuple[str, List[str]]:
    prompt = f"""
    역할: 질의 전처리기
    입력:
      - 질문: "{question}"
      - 최근 대화(최대 3턴): {history}
      - 동적 엔티티 후보(제품/제목): {entities[:100]}
      - 동적 어휘(키워드): {vocab[:200]}
    작업:
      1) 지시어/대명사를 해소하여 구체적인 질문으로 재작성(rewritten).
      2) 검색 키워드 3개 이내 생성(keywords): 쉼표로 구분된 명사 중심.
    출력(JSON):
      {{"rewritten":"...", "keywords":["k1","k2","k3"]}}
    """
    out = llm_json(prompt, max_tokens=260)
    rewritten = (out.get("rewritten") or question).strip()
    kws = [k for k in (out.get("keywords") or []) if k]
    return rewritten, kws[:3]

# 컨텍스트 빌드
def build_context(merged: List[Tuple[str,float,Dict]], *, max_chars: int = 3000, top_n: int = 5) -> str:
    parts, used = [], 0
    for i, (cid, score, meta) in enumerate(merged[:top_n], 1):
        title = (meta or {}).get("title", "N/A")
        text = (meta or {}).get("text_content", "") or ""
        if not text:
            continue
        remain = max_chars - used
        if remain <= 0:
            break
        snippet = text[:max(0, remain-50)] + ("..." if len(text) > remain else "")
        parts.append(f"[자료 {i}] 제목: {title}\n내용: {snippet}\n")
        used += len(parts[-1])
    return "\n".join(parts)

# LLM 답변 생성
def generate_answer_llm(question: str, context: str, citations: List[Tuple[str,float,Dict]], aggregate: Dict | None, *, temperature: float = 0.2, max_tokens: int = 500) -> str:
    if not _openai_client or not context.strip():
        return "관련 컨텍스트를 찾지 못했습니다."
    system = "유니베라 문서 컨텍스트에만 기반해 간결하게 답변하세요. 추측/환각 금지. 한국어 사용. 하단에 출처 표기."
    user = f"참고자료:\n{context}\n\n질문: {question}"
    r = _openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"system","content":system},{"role":"user","content":user}],
        temperature=temperature,
        max_tokens=max_tokens
    )
    ans = r.choices[0].message.content
    cites = []
    for cid, _, meta in citations[:5]:
        cites.append(f"- {(meta or {}).get('title','N/A')} ({cid})")
    if aggregate:
        ans = f"{ans}\n\n집계결과: {aggregate['count']}개 (기준: {aggregate['basis']})"
    return f"{ans}\n\n출처:\n" + "\n".join(cites)

# 세션 메모리 (최근 3턴)
_history: List[Tuple[str,str,str]] = []  # (question, answer, main_entity)
_history_deque = deque(maxlen=3)

# 메인 질의 처리 (한 턴)
def chat_once(question: str, *, vec_w: float = 0.7, bm25_w: float = 0.3, top_k: int = 50, ctx_n: int = 5, max_ctx_chars: int = 3000, debug: bool = True) -> str:
    if debug:
        print("="*90)
        print(f"[질문] {question}")
    vocab, entities = collect_vocab_entities(sample_k=200)
    intent = classify_intent_llm(question, list(_history_deque))
    rewritten, kws = rewrite_and_expand_llm(question, list(_history_deque), vocab, entities)
    if debug:
        print(f"[의도] {intent}")
        print(f"[재작성] {rewritten}")
        print(f"[키워드] {kws}")

    # 벡터 후보 및 BM25 후보 스코어 계산 (디버그 노출용)
    vec_cands = vector_search(rewritten, top_k=top_k)
    bm25_scores = bm25_over_candidates(rewritten, vec_cands)

    # 상위 노출용 정렬
    vec_top = sorted(vec_cands, key=lambda x: x[1], reverse=True)[:10]
    bm25_top = sorted([(cid, bm25_scores.get(cid, 0.0), meta) for (cid, _v, meta) in vec_cands], key=lambda x: x[1], reverse=True)[:10]

    if debug:
        print("-"*90)
        print("[Vector Top]")
        for i, (cid, vscore, meta) in enumerate(vec_top, 1):
            t = (meta or {}).get('title','N/A')
            s = (meta or {}).get('source_doc','')
            print(f"  {i:>2}. {vscore:.4f} | {t} | {cid} | {s}")
        print("-"*90)
        print("[BM25 Top (over vector candidates)]")
        for i, (cid, bscore, meta) in enumerate(bm25_top, 1):
            t = (meta or {}).get('title','N/A')
            s = (meta or {}).get('source_doc','')
            print(f"  {i:>2}. {bscore:.4f} | {t} | {cid} | {s}")

    # 하이브리드 병합
    merged = []
    for cid, vscore, meta in vec_cands:
        b = bm25_scores.get(cid, 0.0)
        final = vec_w*float(vscore) + bm25_w*float(b)
        merged.append((cid, final, meta))
    merged.sort(key=lambda x: x[1], reverse=True)

    if not merged:
        return "관련 정보를 찾지 못했습니다."

    if debug:
        print("-"*90)
        print("[Merged Top]")
        for i, (cid, fscore, meta) in enumerate(merged[:10], 1):
            t = (meta or {}).get('title','N/A')
            s = (meta or {}).get('source_doc','')
            print(f"  {i:>2}. {fscore:.4f} | {t} | {cid} | {s}")

    aggregate = maybe_aggregate_count(intent, merged, question)
    context = build_context(merged, max_chars=max_ctx_chars, top_n=ctx_n)

    if debug:
        print("-"*90)
        print(f"[컨텍스트 길이] {len(context)}")
        print("="*90)

    answer = generate_answer_llm(question, context, merged, aggregate)

    # 메인 엔티티 추출(LLM)에 위임하여 히스토리 저장
    main_ent = ""
    try:
        ent_json = llm_json(f"질문에서 핵심 제품/엔티티 1개를 추출해 JSON만 출력하세요: {{\"entity\":\"...\"}}\n질문: {rewritten}", max_tokens=60)
        main_ent = (ent_json.get("entity") or "").strip()
    except Exception:
        main_ent = ""
    _history_deque.append((question, answer, main_ent))
    return answer

print("LLM 기반 RAG 챗봇 모드입니다. (exit로 종료)")
while True:
    try:
        q = input("\n❓ 질문: ").strip()
        if q.lower() in ["exit","quit","종료"]:
            print("👋 종료합니다.")
            break
        if not q:
            continue
        print("\n처리 중...⏳")
        out = chat_once(q, vec_w=0.7, bm25_w=0.3, top_k=50, ctx_n=5, max_ctx_chars=3000, debug=True)
        print("\n💬 답변:\n" + out)
    except KeyboardInterrupt:
        print("\n👋 종료합니다.")
        break
    except Exception as e:
        print(f"❌ 오류: {e}")


LLM 기반 RAG 챗봇 모드입니다. (exit로 종료)

처리 중...⏳
[질문] 진세노 어쩌고 들은 제품 이썽?
[의도] normal
[재작성] 진세노 관련 제품이 있나요?
[키워드] ['진세노', '제품', '관련']
------------------------------------------------------------------------------------------
[Vector Top]
   1. 0.8195 | 흑삼기력 프리미엄 | heugsamgiryeogpeurimieom-chunk-2 | 흑삼기력프리미엄.md
   2. 0.8043 | 흑삼기력 프리미엄 | doc-chunk-1 | 흑삼기력프리미엄.md
   3. 0.8007 | 흑삼기력 프리미엄 | doc-chunk-0 | 흑삼기력프리미엄.md
   4. 0.8000 | 알로엔 디오리진 수딩스틱 | alroen_diorijin_sudingseutig-chunk-1 | 알로엔_디오리진_수딩스틱.md
   5. 0.7989 | 알로엔 디오리진 스킨케어 100 | alroen_diorijin_seukinkeeo100-chunk-1 | 알로엔_디오리진_스킨케어100.md
   6. 0.7986 | 흑삼기력 데일리 | heugsamgiryeogdeilri-chunk-2 | 흑삼기력데일리.md
   7. 0.7951 | 흑삼기력 프리미엄 | heugsamgiryeogpeurimieom-chunk-0 | 흑삼기력프리미엄.md
   8. 0.7936 | 수면:온 | sumyeonon-chunk-7 | 수면온.md
   9. 0.7932 | 수면:온 | sumyeonon-chunk-8 | 수면온.md
  10. 0.7931 | 흑삼기력 데일리 | heugsamgiryeogdeilri-chunk-4 | 흑삼기력데일리.md
------------------------------------------------------------------------------------------
[BM25 Top 